In [3]:
import sqlite3
import pandas as pd
import dateutil.parser
from datetime import datetime
from time import sleep
from random import randint
from bs4 import BeautifulSoup
import requests
import pandas as pd

from torrequest import TorRequest
from stem import Signal
from stem.control import Controller
import sys

In [64]:
conn = sqlite3.connect('news.db')
c = conn.cursor()

In [15]:
symbol_list = ['nflx']

In [65]:
try:
    c.execute("CREATE TABLE `nasdaq_stock_news` (`date_UTC` TEXT, `title` TEXT, `publisher` TEXT, `url` TEXT, `unix_timestamp` REAL,`symbol` TEXT)")
except:
    print("Error")

In [66]:
conn.commit()

In [125]:
with Controller.from_port(port = 9051) as controller:
    controller.authenticate(password='16:6A38BF1B14C1F2A660DF93C6DDD45B90B340D7DFF4BBB5AE03FB1F3DE4')
    print("Success!")
    controller.signal(Signal.NEWNYM)
    print("New Tor connection processed")
    tr=TorRequest(password='WTKv36s')
    tr.reset_identity() #Reset Tor
    response= tr.get('http://ipecho.net/plain')
    print("New Ip Address",response.text)

Success!
New Tor connection processed
New Ip Address 192.42.116.22


In [93]:
def get_connection(links_site, torR):
    
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

#     with Controller.from_port(port = 9051) as controller:
#         controller.authenticate(password='16:6A38BF1B14C1F2A660DF93C6DDD45B90B340D7DFF4BBB5AE03FB1F3DE4')
#         print("Success!")
#         controller.signal(Signal.NEWNYM)
#         print("New Tor connection processed")
#         tr=TorRequest(password='WTKv36s')
#         tr.reset_identity() #Reset Tor
#         response= tr.get('http://ipecho.net/plain')
#         print("New Ip Address",response.text)
        
    for i in range(15):
        try:
            resp = torR.get(links_site, headers=headers, timeout=5)
            return resp
        except requests.exceptions.Timeout:
            print("Reconnect" + str(i+1))
            with Controller.from_port(port = 9051) as controller:
                controller.authenticate(password='16:6A38BF1B14C1F2A660DF93C6DDD45B90B340D7DFF4BBB5AE03FB1F3DE4')
                print("Success!")
                controller.signal(Signal.NEWNYM)
                print("New Tor connection processed")
                torR=TorRequest(password='WTKv36s')
                torR.reset_identity() #Reset Tor
                response= torR.get('http://ipecho.net/plain')
                print("New Ip Address",response.text)
            pass

    return resp

In [90]:
'''Generalized function to get all news-related articles from a Nasdaq webpage'''
def get_news_urls(links_site, symbol, torRe):
    '''scrape the html of the site'''

    data_l = []
    
    print(links_site)
    
    re = get_connection(links_site, torRe)
        
    html = re.content

    '''convert html to BeautifulSoup object'''
    soup = BeautifulSoup(html , 'lxml')

    '''get list of all links on webpage'''
    #links = soup.find_all('a')
    links = soup.find_all(target="_self")

    headlines_row = soup.find_all('small')

    headlines_row = headlines_row[1:11]

    timestamp_publisher = []
    for i in range(len(headlines_row)):
        timestamp_publisher.append(headlines_row[i].getText())

    titles = [title.getText() for title in links]
    titles = titles[1:]

    urls = [link.get('href') for link in links]
    urls = [url for url in urls if url is not None]

    '''Filter the list of urls to just the news articles'''
    news_urls = [url for url in urls if '/article/' in url]

    datePublished = []
    timestamp = []
    publisher = []

    for i in timestamp_publisher:
        datePublished_temp = dateutil.parser.parse(i.strip().split(sep=" - ")[0]).strftime('%Y-%m-%d %H:%M:%S')
        datePublished.append(datePublished_temp)
        timestamp.append(dateutil.parser.parse(i.strip().split(sep=" - ")[0]).timestamp())
        publisher_temp = i.strip().split(sep=" - ")[1]
        publisher.append(publisher_temp.strip())

    for i in range(len(timestamp)):
        data_l.append({'titles':titles[i], 'datePublished':datePublished[i], 'publisher':publisher[i], 'url':news_urls[i], 'UTC_timestamp':timestamp[i], 'symbol':symbol})
        #insert_data_to_sqlite(datePublished[i], titles[i], publisher[i], news_urls[i], timestamp[i], symbol)
        
    return data_l

In [9]:
def insert_data_to_sqlite(datePublished, titles, publisher, news_urls, unix_timestamp, symbol):
    c.execute("INSERT INTO nasdaq_stock_news (date_UTC, title, publisher, url, unix_timestamp, symbol) VALUES (?, ?, ?, ?, ?, ?)", (datePublished, titles, publisher, news_urls, unix_timestamp, symbol))
    conn.commit()

In [10]:
def get_last_page(links_site, torRe):
    re = get_connection(links_site, torRe)

    html = re.content

    '''convert html to BeautifulSoup object'''
    soup = BeautifulSoup(html , 'lxml')
    
    last_page_link = soup.find(id="quotes_content_left_lb_LastPage").get('href')
    
    page_num = int(last_page_link.split('=')[1])
        
    return page_num

In [11]:
def scrape_all_headlines(symbol, last_page, start_page, tr):
    
    start_page = start_page + 1
    
    data_set = []
    
    for i in range(last_page):
        landing_site = 'http://www.nasdaq.com/symbol/' + str(symbol) + '/news-headlines?page=' + str(start_page)
        data_temp = get_news_urls(landing_site , symbol, tr)
        data_set = data_set + data_temp
        start_page = start_page + 1
        
    return data_set

In [32]:
def check_latest_news():#symbol):
    c = []
    #df_result = pd.read_sql_query("select * from nasdaq_stock_news where symbol = '" + str(symbol) + "'order by datePublished desc limit 5;", conn)
    df_result = pd.read_sql_query("select * from nasdaq_stock_news", conn)
    for i in range(len(df_result)):
        c.append({'symbol':df_result['symbol'].values[i], 'publisher':df_result['publisher'].values[i], 'title':df_result['title'].values[i], 'date_UTC':df_result['date_UTC'].values[i], 'url':df_result['url'].values[i], 'unix_timestamp':df_result['unix_timestamp'].values[i]})
    return c

In [82]:
def check_latest_news(symbol):
    d = {}
    df_result = pd.read_sql_query("select * from nasdaq_stock_news where symbol = '" + str(symbol) + "' order by date_UTC desc;", conn)
    d['symbol'] = df_result['symbol'].values[0]
    d['publisher'] = df_result['publisher'].values[0]
    d['title'] = df_result['title'].values[0]
    d['date_UTC'] = df_result['date_UTC'].values[0]
    d['url'] = df_result['url'].values[0]
    d['unix_timestamp'] = df_result['unix_timestamp'].values[0]
    return d

In [16]:
page = 0

full_data_list = []

for i in symbol_list:
    landing_site = 'http://www.nasdaq.com/symbol/' + i + '/news-headlines'
    last_page = get_last_page(landing_site, tr)

    temp_list = scrape_all_headlines(i, last_page, 31, tr)
    full_data_list = full_data_list + temp_list
    

http://www.nasdaq.com/symbol/nflx/news-headlines?page=32
http://www.nasdaq.com/symbol/nflx/news-headlines?page=33
http://www.nasdaq.com/symbol/nflx/news-headlines?page=34
http://www.nasdaq.com/symbol/nflx/news-headlines?page=35
http://www.nasdaq.com/symbol/nflx/news-headlines?page=36
http://www.nasdaq.com/symbol/nflx/news-headlines?page=37
http://www.nasdaq.com/symbol/nflx/news-headlines?page=38
http://www.nasdaq.com/symbol/nflx/news-headlines?page=39
http://www.nasdaq.com/symbol/nflx/news-headlines?page=40
http://www.nasdaq.com/symbol/nflx/news-headlines?page=41
http://www.nasdaq.com/symbol/nflx/news-headlines?page=42
http://www.nasdaq.com/symbol/nflx/news-headlines?page=43
http://www.nasdaq.com/symbol/nflx/news-headlines?page=44
http://www.nasdaq.com/symbol/nflx/news-headlines?page=45
http://www.nasdaq.com/symbol/nflx/news-headlines?page=46
http://www.nasdaq.com/symbol/nflx/news-headlines?page=47
http://www.nasdaq.com/symbol/nflx/news-headlines?page=48
http://www.nasdaq.com/symbol/nf

In [ ]:
checking = []
db_lastest = check_latest_news('amzn')


while(checking == []):
    temp_dict = get_news_urls(landing_site , symbol, tr)
    for i in range(len(temp_dict)):
        if(db_lastest['unix_timestamp'] < temp_dict['UTC_timestamp'][i]):
            checking
            

In [50]:
df = pd.DataFrame(check_latest_news())

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4868 entries, 0 to 4867
Data columns (total 6 columns):
date_UTC          4868 non-null object
publisher         4868 non-null object
symbol            4868 non-null object
title             4868 non-null object
unix_timestamp    4868 non-null float64
url               4868 non-null object
dtypes: float64(1), object(5)
memory usage: 228.3+ KB


In [52]:
df

,date_UTC,publisher,symbol,title,unix_timestamp,url
0,2019-02-01 23:13:00,Zacks.com,amzn,Disney (DIS) Q1 Earnings Preview: Media Networ...,1.549034e+09,https://www.nasdaq.com/article/disney-dis-q1-e...
1,2019-02-01 22:03:40,Motley Fool,amzn,Stock Market Today: Amazon Gives Disappointing...,1.549030e+09,https://www.nasdaq.com/article/stock-market-to...
2,2019-02-01 21:45:09,NASDAQ.com News,amzn,Market Close Report: NASDAQ Composite index cl...,1.549029e+09,https://www.nasdaq.com/article/market-close-re...
3,2019-02-01 21:41:34,MT Newswires,amzn,Close Update: Major US Stock Indexes Post Smal...,1.549028e+09,https://www.nasdaq.com/article/close-update-ma...
4,2019-02-01 21:32:52,Schaeffer's Research,amzn,Dow Pares Gains But Grabs Another Weekly Win,1.549028e+09,https://www.nasdaq.com/article/dow-pares-gains...
5,2019-02-01 21:11:00,Zacks.com,amzn,What to Expect from Google Parent Alphabet's (...,1.549027e+09,https://www.nasdaq.com/article/what-to-expect-...
6,2019-02-01 21:04:00,Motley Fool,amzn,Why Amazon.com Stock Dropped Today,1.549026e+09,https://www.nasdaq.com/article/why-amazoncom-s...
7,2019-02-01 20:57:16,MT Newswires,amzn,US Stocks Poised to End Week Higher as Indexes...,1.549026e+09,https://www.nasdaq.com/article/us-stocks-poise...
8,2019-02-01 20:52:45,MT Newswires,amzn,"Consumer Sector Update for 02/01/2019: SPR,YUM...",1.549026e+09,https://www.nasdaq.com/article/consumer-sector...
9,2019-02-01 20:21:50,InvestorPlace Media,amzn,"5 Must-See Stock Charts for Monday: AMZN, GE, ...",1.549024e+09,https://www.nasdaq.com/article/5-must-see-stoc...


In [53]:
df2 = df.drop_duplicates()

In [60]:
df2 = df2.reset_index()

In [61]:
df2 = df2.drop(labels=['index'], axis=1)

In [63]:
df2.to_csv('headlines.csv', encoding='utf-8')

In [72]:
def insert_data_to_sqlite(df, length):
    for i in range(length):
        c.execute("INSERT INTO nasdaq_stock_news (date_UTC, title, publisher, url, unix_timestamp, symbol) VALUES (?, ?, ?, ?, ?, ?)", (df['date_UTC'][i], df['title'][i], df['publisher'][i], df['url'][i], df['unix_timestamp'][i], df['symbol'][i]))
        conn.commit()


In [73]:
insert_data_to_sqlite(df2, df2.shape[0])

In [74]:
df3 = pd.DataFrame(check_latest_news())


In [121]:
c = check_latest_news('goog')

In [122]:
c

{'symbol': 'goog',
 'publisher': 'Trefis',
 'title': 'Can Microsoft Sustain Its Growth Momentum Post-Q2?',
 'date_UTC': '2019-01-31 16:34:46',
 'url': 'https://www.nasdaq.com/article/can-microsoft-sustain-its-growth-momentum-post-q2-cm1090878',
 'unix_timestamp': 1548923686.0}

In [126]:
#landing_site = 'http://www.nasdaq.com/symbol/' + 'fb' + '/news-headlines?page=' + str(start_page)

de = get_news_urls('http://www.nasdaq.com/symbol/goog/news-headlines?page=1','fb',tr)



http://www.nasdaq.com/symbol/goog/news-headlines?page=1
Reconnect1
Success!
New Tor connection processed
New Ip Address 185.32.124.226


In [127]:
de

[{'titles': "Alphabet's Waymo To Build Self-driving Cars In Michigan",
  'datePublished': '2019-01-22 21:20:00',
  'publisher': 'RTT News',
  'url': 'https://www.nasdaq.com/article/alphabets-waymo-to-build-selfdriving-cars-in-michigan-20190122-01402',
  'UTC_timestamp': 1548163200.0,
  'symbol': 'fb'},
 {'titles': 'French Regulator Uses New EU Privacy Law To Fine Google $57 Mln',
  'datePublished': '2019-01-21 20:31:00',
  'publisher': 'RTT News',
  'url': 'https://www.nasdaq.com/article/french-regulator-uses-new-eu-privacy-law-to-fine-google-57-mln-20190121-00535',
  'UTC_timestamp': 1548073860.0,
  'symbol': 'fb'},
 {'titles': 'Alphabet Sued By Shareholder Over Payouts To Execs Accused Of Misconduct',
  'datePublished': '2019-01-10 20:50:00',
  'publisher': 'RTT News',
  'url': 'https://www.nasdaq.com/article/alphabet-sued-by-shareholder-over-payouts-to-execs-accused-of-misconduct-20190110-01058',
  'UTC_timestamp': 1547124600.0,
  'symbol': 'fb'},
 {'titles': "Google's New Policy Ch

In [112]:
for i in range(len(de)):
    #print(de[i]['titles'])
    if((c['unix_timestamp']) > de[i]['UTC_timestamp']):
        print("yes")

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
